## L2Beat

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_l2beat import AdapterL2Beat

adapter_params = {
}
load_params = {
    'origin_keys' : None,
    #'origin_keys' : ['optimism','arbitrum'] # see all options in adapter_mapping.py 
}

# initialize adapter
ad = AdapterL2Beat(adapter_params, db_connector)
# extract
df= ad.extract(load_params)
# load
ad.load(df)

## DefiLlama

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_defillama import AdapterDefiLlama

adapter_params = {
}
load_params = {
    'origin_keys' : None,
    #'origin_keys' : ['ethereum'] # see all options in adapter_mapping.py
}

# initialize adapter
ad = AdapterDefiLlama(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## Coingecko

run for projects / chains

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_coingecko import AdapterCoingecko

adapter_params = {
}
load_params = {
    'load_type' : 'project',
    'metric_keys' : ['price', 'volume', 'market_cap'],
    'origin_keys' : None,
    #'origin_keys' : ['aptos'], # see all options in adapter_mapping.py
    'days' : 'auto', # auto, max, or a number (as string)
    'vs_currencies' : ['usd', 'eth']
}

# initialize adapter
ad = AdapterCoingecko(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

and for imx tokens

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_coingecko import AdapterCoingecko

adapter_params = {
}
load_params = {
    'load_type' : 'imx_tokens'
}

# initialize adapter
ad = AdapterCoingecko(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## Dune

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_dune import AdapterDune

adapter_params = {
    'api_key' : os.getenv("DUNE_API")
}

load_params = {
    # 'query_names' : ['waa'], ## fundamentals, waa, stables_mcap
    # 'days' : 100,
    'query_names' : None,
    'days' : 'auto',
}

# initialize adapter
ad = AdapterDune(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# upload
ad.load(df)

## Flipside
sometimes some Flipside queries just get stuck -- gotta retrigger them

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_flipside import AdapterFlipside

adapter_params = {
    'api_key' : os.getenv("FLIPSIDE_API")
}
load_params = {
    'origin_keys' : ['zksync_era'],
    'metric_keys' : ['stables_mcap'],
    'days' : 'auto',
    # 'origin_keys' : None,
    # 'metric_keys' : None,
    # 'days' : 'auto',
}

# initialize adapter
ad = AdapterFlipside(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

In [ ]:
## testing the new flipside api

from flipside import Flipside
flipside = Flipside("74c43ebc-3291-4953-8aeb-65640da7c852", "https://api-v2.flipsidecrypto.xyz")

sql = """
 select 
    BLOCK_NUMBER, BLOCK_TIMESTAMP, BLOCK_HASH, TX_HASH, NONCE, POSITION, ORIGIN_FUNCTION_SIGNATURE, FROM_ADDRESS, TO_ADDRESS, ETH_VALUE, TX_FEE, GAS_PRICE, GAS_LIMIT, 
    GAS_USED, L1_GAS_PRICE, L1_GAS_USED, L1_FEE_SCALAR, L1_SUBMISSION_BATCH_INDEX, L1_SUBMISSION_TX_HASH, L1_STATE_ROOT_BATCH_INDEX, 
    L1_STATE_ROOT_TX_HASH, CUMULATIVE_GAS_USED, INPUT_DATA, STATUS
from optimism.core.fact_transactions
where block_number >= 104130000
and block_number < 104135000
order by block_number asc
"""

# Run the query against Flipside's query engine and await the results
query_result_set = flipside.query(sql)

## Zettablock

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_zettablock import AdapterZettablock

adapter_params = {
    'api_key' : os.getenv("ZETTABLOCK_API")
}
load_params = {
    #'origin_keys' : ['zksync_era', 'polygon_zkevm'],
    #'metric_keys' : ['txcount', ''],
    'days' : 10,
    'origin_keys' : None,
    'metric_keys' : None,
    # 'days' : 'auto',
}

# initialize adapter
ad = AdapterZettablock(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## ZettaBlock raw data load

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_zettablock import AdapterZettaBlockRaw

adapter_params = {
    'api_key' : os.getenv("ZETTABLOCK_API")
}

load_params = {
    #'keys' : ['polygon_zkevm_tx', 'zksync_era_tx'],
    'keys' : ['zksync_era_tx'],
    #'block_start' : 'auto', ## 'auto' or a block number as int
    'block_start' : 9137631, ## 'auto' or a block number as int
}

# initialize adapter
ad = AdapterZettaBlockRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params, if_exists = 'ignore')

## Chainbase raw

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_chainbase import AdapterChainbaseRaw

adapter_params = {
    'api_key' : os.getenv("CHAINBASE_API")
}

load_params = {
    'keys' : ['arbitrum_tx'],
    #'block_start' : 'auto', ## 'auto' or a block number as int
    'block_start' : 64900000, ## until 65,570,000
}

# initialize adapter
ad = AdapterChainbaseRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params)

## IMX raw data load

In [ ]:
## implement orchestration?

from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_imx import AdapterRawImx

adapter_params = {
    'load_types' : ['withdrawals', 'deposits', 'trades', 'orders_filled', 'transfers', 'mints'],
    'forced_refresh' : 'no',

    #'load_types' : ['orders_filled'],
    #'forced_refresh' : '2023-04-01T00:00:00.000000Z',
}

# initialize adapter
ad = AdapterRawImx(adapter_params, db_connector)
# extract raw (and load raw in case of IMX)
df_raw = ad.extract_raw()

## Flipside raw data load

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_flipside import AdapterFlipsideRaw

adapter_params = {
    'api_key' : os.getenv("FLIPSIDE_API")
}

load_params = {
    'keys' : ['arbitrum_tx', 'optimism_tx'],
    'block_start' : 'auto',
}

# initialize adapter
ad = AdapterFlipsideRaw(adapter_params, db_connector)
# extract
df = ad.extract_raw(load_params)

## SQL aggregation

In [ ]:
## Loads currently IMX txcount, daa, fees_paid
## also loads user_base_weekly

from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'metrics', ## usd_to_eth or metrics
    
    #'days' : 'auto', ## days as int our 'auto
    'origin_keys' : None, ## origin_keys as list or None
    #'metric_keys' : None, ## metric_keys as list or None

    'days' : '28', ## days as int our 'auto
    #'origin_keys' : ['imx'], ## origin_keys as list or None
    'metric_keys' : ['user_base_weekly'], ## metric_keys as list or None
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
df = ad.extract(load_params)
# # load
ad.load(df)

In [ ]:
df.origin_key.unique()

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'usd_to_eth', ## usd_to_eth or metrics
    'days' : 180, ## days as int
    'origin_keys' : None, ## origin_keys as list or None
    'metric_keys' : None, ## metric_keys as list or None
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
df = ad.extract(load_params)
# # load
ad.load(df)

### Blockspace logic
- for each chain, aggregate the daily contracts usage


In [6]:
from src.db_connector import DbConnector
db_connector = DbConnector()

chain_list = ['arbitrum', 'zksync_era', 'polygon_zkevm']
chain_list = ['arbitrum']
days = 3000

In [7]:
for chain in chain_list:
    ## aggregate contract data
    print(f"aggregating contract data for {chain} and last {days} days...")
    df = db_connector.get_blockspace_contracts(chain, days)
    df.set_index(['address', 'date', 'origin_key'], inplace=True)

    print(f"upserting contract data for {chain}. Total rows: {df.shape[0]}...")
    db_connector.upsert_table('blockspace_fact_contract_level', df)

    ## aggregate native transfers
    print(f"aggregating native_transfers for {chain} and last {days} days...")
    df = db_connector.get_blockspace_native_transfers(chain, days)
    df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    print(f"upserting native_transfers for {chain}. Total rows: {df.shape[0]}...")
    db_connector.upsert_table('blockspace_fact_sub_category_level', df)


    # ALL below needs to be retriggerd when mapping changes (e.g. new addresses got labeled or new categories added etc.)
    ## aggregate by sub categories
    print(f"aggregating sub categories for {chain} and last {days} days...")
    df = db_connector.get_blockspace_sub_categories(chain, days)
    df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    print(f"upserting sub categories for {chain}. Total rows: {df.shape[0]}...")
    db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    ## determine unlabeled usage
    print(f"aggregating unlabeled usage for {chain} and last {days} days...")
    df = db_connector.get_blockspace_unlabeled(chain, days)
    df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    print(f"upserting unlabeled usage for {chain}. Total rows: {df.shape[0]}...")
    db_connector.upsert_table('blockspace_fact_sub_category_level', df)

aggregating contract data for arbitrum and last 3000 days...
upserting contract data for arbitrum. Total rows: 1060083...
Batch upload necessary. Total size: 1060083
Batch 100000
Batch 200000
Batch 300000
Batch 400000
Batch 500000
Batch 600000
Batch 700000
Batch 800000
Batch 900000
Batch 1000000
Batch 1100000
aggregating native_transfers for arbitrum and last 3000 days...
upserting native_transfers for arbitrum. Total rows: 461...
aggregating sub categories for arbitrum and last 3000 days...
upserting sub categories for arbitrum. Total rows: 6819...
aggregating unlabeled usage for arbitrum and last 3000 days...
upserting unlabeled usage for arbitrum. Total rows: 461...


In [ ]:
## df order by block_number asc
df.sort_values(by=['date'], inplace=True, ascending=True)

In [ ]:
df.shape

In [ ]:
## take first 50000 rows from df and store in df1
df1 = df.iloc[:50000]

In [ ]:
df1.set_index(['address', 'date', 'origin_key'], inplace=True)

In [ ]:
print(f"upserting contract data for {chain}. Total rows: {df1.shape[0]}...")
db_connector.upsert_table('blockspace_fact_contract_level', df1)

## S3

In [ ]:
## connect to s3 bucket and output list of files
import pandas as pd
import boto3
import os

s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
)

bucket = s3.Bucket('gtp-longterm')

## get list of files in bucket with last modified date
files = []
for obj in bucket.objects.all():
    files.append([obj.key, obj.last_modified])

df = pd.DataFrame(files, columns=['key', 'last_modified'])

## filter out files where key starts with 'imx'
df = df[~df.key.str.startswith('imx')]

df['chain'] = df.key.str.split('/').str[0]

## create new column block_range that extracts the string between 'tx_' and '.parquet' in the key column using lambda function
df['block_range'] = df.key.apply(lambda x: x[x.find('tx_')+3:x.find('.parquet')])
df['block_start'] = df.block_range.str.split('-').str[0].astype(int)
df['block_end'] = df.block_range.str.split('-').str[1].astype(int)

## sort by block_start
df.sort_values(by='block_start', inplace=True, ascending=True)

In [ ]:
df

### Arbitrum

In [ ]:
df_arbitrum = df[df.chain == 'arbitrum']

In [ ]:
df_arbitrum

In [ ]:
## load first file in df_arbitrum into df
df_arbitrum_flipside = pd.read_parquet(f"s3://gtp-longterm/{df_arbitrum.key.iloc[0]}")
df_arbitrum_flipside.sort_values(by='ETH_VALUE', inplace=True, ascending=False)
df_arbitrum_flipside.head(10)

In [ ]:
## load first file in df_arbitrum with block_start >= 96528665 into df
df_arbitrum_chainbase = pd.read_parquet(f"s3://gtp-longterm/{df_arbitrum[df_arbitrum.block_start >= 96528665].key.iloc[10]}")
df_arbitrum_chainbase.head()

### Optimism

In [ ]:
df_optimism = df[df.chain == 'optimism']

In [ ]:
## load first file in df_arbitrum into df
df_optimism_flipside = pd.read_parquet(f"s3://gtp-longterm/{df_optimism.key.iloc[0]}")
df_optimism_flipside.head()

In [ ]:
# df_optimism_flipside[['TX_HASH', 'ETH_VALUE', 'TX_FEE', 'GAS_PRICE', 'GAS_LIMIT', 'GAS_USED',
#        'L1_GAS_PRICE', 'L1_GAS_USED', 'L1_FEE_SCALAR',  'CUMULATIVE_GAS_USED' ]]

## filter df_optimism_flipside where TX_HASH = 0xA5E0DACD8D29020C04139F8506C01CBA8B6E561CE567DF8DA35857722232F559 and select columns 'TX_HASH', 'ETH_VALUE', 'TX_FEE', 'GAS_PRICE', 'GAS_LIMIT', 'GAS_USED','L1_GAS_PRICE', 'L1_GAS_USED', 'L1_FEE_SCALAR',  'CUMULATIVE_GAS_USED' 

df_optimism_flipside[df_optimism_flipside.TX_HASH == '0xA5E0DACD8D29020C04139F8506C01CBA8B6E561CE567DF8DA35857722232F559'.lower()][['TX_HASH', 'ETH_VALUE', 'TX_FEE', 'GAS_PRICE', 'GAS_LIMIT', 'GAS_USED','L1_GAS_PRICE', 'L1_GAS_USED', 'L1_FEE_SCALAR',  'CUMULATIVE_GAS_USED' ]]

In [ ]:
df_optimism_flipside['GAS_PRICE'] / 1e9

In [ ]:
df_optimism_flipside.columns

In [ ]:
## load first file in df_arbitrum with block_start >= 96528665 into df
df_optimism_chainbase = pd.read_parquet(f"s3://gtp-longterm/{df_optimism[df_optimism.block_start >= 103428989].key.iloc[0]}")
df_optimism_chainbase.head()

### Polygon zkEVM

In [ ]:
df_polygon = df[df.chain == 'polygon_zkevm']

In [ ]:
## load first file in df_arbitrum into df
df_polygon_zb = pd.read_parquet(f"s3://gtp-longterm/{df_polygon.key.iloc[0]}")
df_polygon_zb.head()

### zkSync Era

In [ ]:
df_zksync = df[df.chain == 'zksync_era']

In [ ]:
## load first file in df_arbitrum into df
df_zksync_zb = pd.read_parquet(f"s3://gtp-longterm/{df_zksync.key.iloc[0]}")
df_zksync_zb.head()

In [ ]:
# ## rename files in S3 bucket that contain 'block_'
# for index, row in df[df.key.str.contains('block_')].iterrows():
#     print(row['key'])
#     old_key = row['key']
#     new_key = old_key.replace('block_', '')
#     print(new_key)
#     s3.Object('gtp-longterm', new_key).copy_from(CopySource='gtp-longterm/'+old_key)
#     #s3.Object('gtp-longterm', old_key).delete()


In [ ]:
df_arbitrum[df_arbitrum.block_start >= 96528665].sort_values(by='last_modified', ascending=True)

## load polygon zkevm data from S3

In [ ]:
import boto3
import pandas as pd
import os
from dotenv import load_dotenv
from io import BytesIO
from sqlalchemy import create_engine, exc
import sys
from pangres import upsert

# Load environment variables
def load_environment():
    load_dotenv()

    # Postgres details from .env file
    db_user = os.getenv("DB_USERNAME")
    db_password = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_database = os.getenv("DB_DATABASE")

    return db_user, db_password, db_host, db_database

def connect_to_s3():
    try:
        aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
        aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
        bucket_name = 'gtp-longterm'

        if not aws_access_key_id or not aws_secret_access_key or not bucket_name:
            raise ValueError("AWS access key ID, secret access key, or bucket name not found in environment variables.")

        s3 = boto3.client(
            's3',
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        )
        return s3, bucket_name
    except Exception as e:
        print("An error occurred while connecting to S3:", str(e))
        return None, None

def get_files_from_bucket(s3, bucket_name, folder_key):
    return s3.list_objects(Bucket=bucket_name, Prefix=folder_key)

def load_parquet_from_s3(s3, bucket_name, file_key):
    print(f"Loading file {file_key} from bucket {bucket_name}...")
    try:
        obj = s3.get_object(Bucket=bucket_name, Key=file_key)
        df = pd.read_parquet(BytesIO(obj['Body'].read()))
        return df
    except Exception as e:
        print(f"Error getting file {file_key} from bucket {bucket_name}.")
        print(e)

def create_db_engine(db_user, db_password, db_host, db_database):
    print("Creating database engine...")
    try:
        # create connection to Postgres
        engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}/{db_database}')
        engine.connect()  # test connection
        return engine
    except exc.SQLAlchemyError as e:
        print("Error connecting to database. Check your database configurations.")
        print(e)
        raise ValueError("Error inserting data into table.")

def insert_into_db(df, engine, table_name):
    # Set the DataFrame's index to 'tx_hash' (primary key)
    df.set_index('tx_hash', inplace=True)
    df.index.name = 'tx_hash'
    
    # Insert data into database
    print(f"Inserting data into table {table_name}...")
    try:
        upsert(engine, df, table_name, if_row_exists='ignore')
        print("Data inserted successfully.")
    except Exception as e:
        print(f"Error inserting data into table {table_name}.")
        print(e)
        print(df)
        raise ValueError("Error inserting data into table.")

def process_files_in_bucket(files_in_bucket, s3, bucket_name, engine, table_name, start_index):
    # if start_index is less than 0 or greater than the number of files, set it to 0
    start_index = max(0, min(start_index, len(files_in_bucket['Contents']) - 1))
    #for file in files_in_bucket['Contents'][:10]:  # Only process first 10 files
    for file in files_in_bucket['Contents'][start_index:]:
        # Print file index in the list
        print(f"--> Processing file {files_in_bucket['Contents'].index(file) + 1} of {len(files_in_bucket['Contents'])}...")
        if file['Key'].endswith('.parquet'):  # to ensure only .parquet files are processed
            # Parse the min and max block numbers from the filename
            filename = os.path.basename(file['Key'])
            min_block, max_block = filename.rsplit('_', 1)[1].split('-')[0], filename.rsplit('_', 1)[1].split('-')[1].split('.')[0]
            print(f"--> Processing file {filename} with block range {min_block} - {max_block}")

            df = load_parquet_from_s3(s3, bucket_name, file['Key'])
            if df is not None:
                print(f"Preparing DataFrame for file {filename}...")
                if table_name == 'polygon_zkevm_tx':
                    df = prepare_dataframe_polygon(df)  # call to the function that prepares the dataframe
                elif table_name == 'zksync_era_tx':
                    df = prepare_dataframe_zksync(df)  # call to the function that prepares the dataframe
                insert_into_db(df, engine, table_name)

def prepare_dataframe_polygon(df):
    print("Preparing DataFrame for polygon_zkevm_tx...")
    # Columns to be used from the dataframe
    cols = ['block_number', 'block_time', 'hash', 'from_address', 'to_address', 'status', 'value', 'gas_limit', 'gas_used', 'gas_price', 'type', 'receipt_contract_address']

    # Filter the dataframe to only include the above columns plus 'input' for calculations
    df = df.loc[:, cols + ['input']]

    # Rename columns
    df.rename(columns={'hash': "tx_hash", "block_time":"block_timestamp"}, inplace=True, errors="ignore")

    # Add tx_fee column
    df['tx_fee'] = df['gas_used'] * df['gas_price']  / 1e18

    # gas_price column: convert to eth
    df['gas_price'] = df['gas_price'].astype(float) / 1e18

    # Add native_transfer column True when (input = 0x OR input is empty) and value > 0 then true else false
    df['value'] = df['value'].astype(float) / 1e18

    # Add empty_input column True when input is empty or 0x then true else false
    df['empty_input'] = df['input'].apply(lambda x: True if (x == '0x' or x == '') else False)

    # status column: 1 if status is success, 0 if failed else -1
    df['status'] = df['status'].apply(lambda x: 1 if x == 1 else 0 if x == 0 else -1)

    # Convert block_time to datetime
    df['block_timestamp'] = pd.to_datetime(df['block_timestamp'])

    # Handle bytea data type
    for col in ['tx_hash', 'to_address', 'from_address', 'receipt_contract_address']:
        if col in df.columns:
            df[col] = df[col].str.replace('0x', '\\x', regex=False)
        else:
            print(f"Column {col} not found in dataframe.")
    
    # Drop the 'input' column
    df = df.drop(columns=['input'])

    return df

def prepare_dataframe_zksync(df):
    print("Preparing DataFrame for zksync_era_tx...")

    # Columns to be used from the dataframe
    cols = ['block_number', 'block_time', 'hash', 'from_address', 'to_address', 'status', 'value', 'gas_limit', 'gas_used', 'gas_price', 'type', 'receipt_contract_address']

    # Filter the dataframe to only include the above columns plus 'input' for calculations
    df = df.loc[:, cols + ['input']]

    # Rename columns
    df.rename(columns={'hash': "tx_hash", "block_time":"block_timestamp"}, inplace=True, errors="ignore")

    # Add tx_fee column
    df['tx_fee'] = df['gas_used'] * df['gas_price']  / 1e18

    # gas_price column: convert to eth
    df['gas_price'] = df['gas_price'].astype(float) / 1e18

    # value column divide by 1e18 to convert to eth
    df['value'] = df['value'].astype(float) / 1e18

    # Add empty_input column True when input is empty or 0x then true else false
    df['empty_input'] = df['input'].apply(lambda x: True if (x == '0x' or x == '') else False)

    # status column: 1 if status is success, 0 if failed else -1
    df['status'] = df['status'].apply(lambda x: 1 if x == 1 else 0 if x == 0 else -1)

    # Convert block_time to datetime
    df['block_timestamp'] = pd.to_datetime(df['block_timestamp'])

    # Handle bytea data type
    for col in ['tx_hash', 'to_address', 'from_address', 'receipt_contract_address']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace('0x', '\\x', regex=False)
        else:
            print(f"Column {col} not found in dataframe.")
    
    # Drop the 'input' column
    df = df.drop(columns=['input'])

    return df


def main_polygon():
    # List of folders in the S3 bucket
    chain_folders = ['arbitrum', 'imx', 'optimism', 'polygon_zkevm', 'zksync_era']
    
    # Load environment variables
    db_user, db_password, db_host, db_database = load_environment()

    # Create DB engine
    engine = create_db_engine(db_user, db_password, db_host, db_database)

    # Connect to S3 bucket
    s3, bucket_name = connect_to_s3()

    # Select the desired chain folder
    folder_key = chain_folders[3]+'/'

    # Get list of all files in the specific S3 bucket and folder
    files_in_bucket = get_files_from_bucket(s3, bucket_name, folder_key)

    # Sort the files by min_block
    files_in_bucket['Contents'].sort(key=lambda x: int(os.path.basename(x['Key']).rsplit('_', 1)[1].split('-')[0]))

    # Table name in the database
    table_name = 'polygon_zkevm_tx'

    # Process the files in the bucket
    process_files_in_bucket(files_in_bucket, s3, bucket_name, engine, table_name, 0)

def main_zksync():
    # List of folders in the S3 bucket
    chain_folders = ['arbitrum', 'imx', 'optimism', 'polygon_zkevm', 'zksync_era']
    
    # Load environment variables
    db_user, db_password, db_host, db_database = load_environment()

    # Create DB engine
    engine = create_db_engine(db_user, db_password, db_host, db_database)

    # Connect to S3 bucket
    s3, bucket_name = connect_to_s3()

    # Select the desired chain folder
    folder_key = chain_folders[4]+'/'

    # Get list of all files in the specific S3 bucket and folder
    files_in_bucket = get_files_from_bucket(s3, bucket_name, folder_key)

    # Sort the files by min_block
    files_in_bucket['Contents'].sort(key=lambda x: int(os.path.basename(x['Key']).rsplit('_', 1)[1].split('-')[0]))

    # Table name in the database
    table_name = 'zksync_era_tx'

    # Process the files in the bucket
    process_files_in_bucket(files_in_bucket, s3, bucket_name, engine, table_name, 981)

In [ ]:
main_zksync()